In [1]:
import transformers

In [2]:
from datasets import load_dataset

In [3]:
from transformers import DataCollatorForWholeWordMask, AutoTokenizer, AutoModelForMaskedLM, Trainer, TrainingArguments, pipeline, BertTokenizerFast, BertTokenizer

In [4]:
from tokenizers import BertWordPieceTokenizer

In [5]:
import requests

In [6]:
import os
import shutil

In [7]:
# MODEL_NAME = "DeepPavlov/rubert-base-cased" # или "DeepPavlov/bert-base-bg-cs-pl-ru-cased"
MODEL_NAME = "DeepPavlov/bert-base-bg-cs-pl-ru-cased"

In [8]:
OUTPUT_DIR = "old_russian_slavic_bert_finetune"

In [9]:
TRAIN_FILE = "final_dataset_ready.txt"

In [10]:
VOCAB_DIR = "./custom_vocab_dir"

In [11]:
ORIGINAL_FILE = "vocab_extended.txt"

In [12]:
VOCAB_SIZE_LIMIT = 5000

In [13]:
vocab_url = "https://huggingface.co/DeepPavlov/bert-base-bg-cs-pl-ru-cased/resolve/main/vocab.txt"
try:
    r = requests.get(vocab_url)
    original_vocab = r.text.splitlines()
    print(f" Downloader.{len(original_vocab)} tokens in the dataset")
except Exception as e:
    raise ValueError(f"Error: {e}")

 Downloader.119547 tokens in the dataset


In [14]:
with open(TRAIN_FILE, 'r', encoding='utf-8') as f:
    text_content = f.read()

In [15]:
existing_vocab_set = set(original_vocab)
tokens_to_append = []

In [16]:
unique_chars = set(text_content)
chars_found = 0

In [17]:
for c in unique_chars:
    if c.strip(): # Пропускаем пробелы
        # 1. Добавляем саму букву (для начала слова)
        if c not in existing_vocab_set and c not in tokens_to_append:
            tokens_to_append.append(c)
            chars_found += 1

        # 2. ВАЖНО: Добавляем вариант с ## (для середины слова)
        sub_c = "##" + c
        if sub_c not in existing_vocab_set and sub_c not in tokens_to_append:
            tokens_to_append.append(sub_c)

In [18]:
print(f"Added letters/symbols: {chars_found} (Examples: {tokens_to_append})")

Added letters/symbols: 129 (Examples: ['ѥ', '##ѥ', '̈', '##̈', '̑', '##̑', '҇', '##҇', '\uf47e', '##\uf47e', '⁞', '##⁞', '\uf13f', '##\uf13f', '̆', '##̆', 'ᲂ', '##ᲂ', '͗', '##͗', 'ͨ', '##ͨ', 'ⱖ', '##ⱖ', 'Ѧ', '##Ѧ', '̾', '##̾', 'Ⱉ', '##Ⱉ', '“', '##“', 'Ѡ', '##Ѡ', 'ⰹ', '##ⰹ', 'ȥ', '##ȥ', 'ꙇ', '##ꙇ', 'ꙃ', '##ꙃ', '\uf480', '##\uf480', 'ѷ', '##ѷ', '͞', '##͞', 'ѫ', '##ѫ', 'ӱ', '##ӱ', 'Ѳ', '##Ѳ', 'ⸯ', '##ⸯ', '\uf2b4', '##\uf2b4', 'ӏ', '##ӏ', 'ҍ', '##ҍ', '꙯', '##꙯', '\uf2b5', '##\uf2b5', '\uf086', '##\uf086', '”', '##”', '\ufeff', '##\ufeff', 'ѧ', '##ѧ', 'ѱ', '##ѱ', '\uf2db', '##\uf2db', '\uf067', '##\uf067', 'ꙗ', '##ꙗ', 'ҁ', '##ҁ', 'ᴤ', '##ᴤ', '⟧', '##⟧', '\uf265', '##\uf265', '͆', '##͆', '҅', '##҅', '´', '##´', 'ꙩ', '##ꙩ', '̋', '##̋', '⋮', '##⋮', '…', '##…', '͛', '##͛', '҆', '##҆', '¦', '##¦', '͠', '##͠', 'Ѿ', '##Ѿ', 'ѽ', '##ѽ', 'Ҍ', '##Ҍ', '\uf467', '##\uf467', 'ѭ', '##ѭ', 'ꙁ', '##ꙁ', 'Ѻ', '##Ѻ', 'ѿ', '##ѿ', '—', '##—', 'ꙙ', '##ꙙ', '\uf245', '##\uf245', '\uf4a4', '##\uf4a4', 'Ѕ', '##Ѕ', 'Ꙇ'

## Adding vocabulary / words

In [19]:
from collections import Counter

In [20]:
words = text_content.split()
word_counts = Counter(words)

In [21]:
forbidden = set(".,;!?:()[]\"'«»-\n\r\t")
words_found = 0

In [22]:
for w, c in word_counts.most_common(VOCAB_SIZE_LIMIT + 2000): # Берем с запасом
    # Условия:
    # 1. Слова нет в словаре
    # 2. Длина > 1 (буквы уже добавили)
    # 3. Нет запрещенных знаков внутри
    if w not in existing_vocab_set and len(w) > 1:
        if not any(bad in w for bad in forbidden):
            tokens_to_append.append(w)
            words_found += 1
            if words_found >= VOCAB_SIZE_LIMIT:
                break

In [23]:
print(f"Adding words: {words_found}")
print(f"All tokens: {len(tokens_to_append)}")

Adding words: 5000
All tokens: 5258


In [24]:
print(f"Creating a file {ORIGINAL_FILE}...")
full_vocab = original_vocab + tokens_to_append

with open(ORIGINAL_FILE, "w", encoding="utf-8") as f:
    for token in full_vocab:
        f.write(token + "\n")

Creating a file vocab_extended.txt...


In [25]:
os.makedirs(VOCAB_DIR, exist_ok=True)
dest_file = os.path.join(VOCAB_DIR, "vocab.txt")
shutil.copy(ORIGINAL_FILE, dest_file)

'./custom_vocab_dir/vocab.txt'

In [26]:
tokenizer = BertTokenizer.from_pretrained(
      VOCAB_DIR,
      do_lower_case=False,
      unk_token="[UNK]"
  )

In [27]:
len(tokenizer)

124805

In [28]:
print("\n>>> 🧪 ТЕСТ ТОКЕНИЗАЦИИ:")
test_phrase = "поклоно ѿ онѳима ко осподину"
tokens = tokenizer.tokenize(test_phrase)
print(f"   Фраза: '{test_phrase}'")
print(f"   Токены: {tokens}")


>>> 🧪 ТЕСТ ТОКЕНИЗАЦИИ:
   Фраза: 'поклоно ѿ онѳима ко осподину'
   Токены: ['поклоно', 'ѿ', 'он', '##ѳ', '##им', '##а', 'ко', 'оспо', '##дин', '##у']


In [29]:
save_path = "./old_rus_tokenizer"
tokenizer.save_pretrained(save_path)

('./old_rus_tokenizer/tokenizer_config.json',
 './old_rus_tokenizer/tokenizer.json')

## Loading everything

In [30]:
tokenizer = AutoTokenizer.from_pretrained("./old_rus_tokenizer", use_fast=True)

In [31]:
model = AutoModelForMaskedLM.from_pretrained("DeepPavlov/bert-base-bg-cs-pl-ru-cased")

Loading weights:   0%|          | 0/204 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie bert.embeddings.word_embeddings.weight to cls.predictions.decoder.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
The tied weights mapping and config for this model specifies to tie cls.predictions.bias to cls.predictions.decoder.bias, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
BertForMaskedLM LOAD REPORT from: DeepPavlov/bert-base-bg-cs-pl-ru-cased
Key                          | Status     |  | 
-----------------------------+------------+--+-
cls.seq_relationship.weight  | UNEXPECTED |  | 
cls.seq_relationship.bias    | UNEXPECTED |  | 
bert.pooler.dense.bias       | UNEXPECTED |  | 
bert.embeddings.position_ids | UNEXPECTED |  | 
bert.pooler.dense.weight     | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:c

In [32]:
model.resize_token_embeddings(len(tokenizer))

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
The new lm_head weights will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Embedding(124805, 768, padding_idx=0)

In [33]:
dataset = load_dataset("text", data_files={"train": TRAIN_FILE})

In [34]:
def tokenize_function(examples):
    # Токенизируем
    result = tokenizer(
        examples["text"],
        truncation=True,
        max_length=512,
        return_special_tokens_mask=True
    )

    # ГЛАВНАЯ МАГИЯ: Добавляем word_ids для каждого примера
    # Это позволяет коллатору понять, какие токены принадлежат одному слову
    all_word_ids = []
    for i in range(len(examples["text"])):
        word_ids = result.word_ids(batch_index=i)
        # Заменяем None (спецтокены) на -1 для совместимости
        processed_word_ids = [w if w is not None else -1 for w in word_ids]
        all_word_ids.append(processed_word_ids)

    result["word_ids"] = all_word_ids
    return result

In [35]:
tokenized_datasets = dataset.map(
    tokenize_function,
    batched=True,
    num_proc=4,        # Используем 4 ядра процессора для ускорения
    remove_columns=["text"] # Удаляем исходный текст, оставляем только цифры
)

Map (num_proc=4):   0%|          | 0/71255 [00:00<?, ? examples/s]

In [36]:
block_size = 512
def group_texts(examples):
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    return result

In [37]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    num_proc=4,
)

Map (num_proc=4):   0%|          | 0/71255 [00:00<?, ? examples/s]

In [38]:
print(f"Total blocks for training: {len(lm_datasets['train'])}")

Total blocks for training: 3961


In [39]:
data_collator = DataCollatorForWholeWordMask(
    tokenizer=tokenizer,
    mlm=True,
    mlm_probability=0.15
)

/usr/local/lib/python3.12/dist-packages/transformers/data/data_collator.py:1028: FutureWarning: DataCollatorForWholeWordMask is deprecated and will be removed in a future version, you can now use DataCollatorForLanguageModeling with whole_word_mask=True instead.
  warnings.warn(


In [40]:
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=15,             # Ставим 15 эпох для надежности
    per_device_train_batch_size=8,   # 8 для T4 GPU (если вылетает OOM, ставь 4)
    gradient_accumulation_steps=2,   # Накапливаем градиент (эффективный батч = 16)
    learning_rate=5e-5,
    weight_decay=0.01,
    save_steps=500,
    save_total_limit=2,
    fp16=True,                       # Включаем Mixed Precision (ускорение)
    logging_steps=50,
    report_to="none",
    remove_unused_columns=False
)

In [41]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=lm_datasets["train"],
    processing_class =tokenizer # Передаем токенизатор, чтобы он сохранился в финале
)

In [42]:
trainer.train()

IndexError: too many indices for array: array is 0-dimensional, but 3 were indexed

In [ ]:
trainer.save_model(OUTPUT_DIR)

In [ ]:
print("\n>>> ТЕСТИРУЕМ МОДЕЛЬ:")
fill_mask = pipeline("fill-mask", model=OUTPUT_DIR, tokenizer=OUTPUT_DIR, device=0)

test_phrase = "поклоно ѿ [MASK] ко осподину"
print(f"Запрос: {test_phrase}")
res = fill_mask(test_phrase)
for r in res:
    print(f"🔹 {r['token_str']} ({r['score']:.1%})")